In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import os
import glob
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
subj2subj_dist = np.load("../preprocessed_data/subj2subj_dist.npz", allow_pickle=True)
subj2vit_dist = np.load("../preprocessed_data/subj2vit_dist.npz", allow_pickle=True)
subj2vit_official_dist = np.load("../preprocessed_data/subj2vit_official_dist.npz", allow_pickle=True)
subj2gbvs_dist = np.load("../preprocessed_data/subj2gbvs_dist.npz", allow_pickle=True)

In [3]:
data_info = pd.read_csv("../30s_segment_description.csv")

In [4]:
data_info

,index,file name,movie name,ch,location in ch (min/frames),length,AR (approx),size,genre (IMDB),genre category,...,human figures (0-5),nature (0-5),man-made objects (0-5),environment,light (0-5),light category,aud. Info,possible question,possible answers,content elements
0,NaN,STEPF_3a,The Stepfather,3,00:33.03/792-01:03.05/1511,30.01s,21x9,853x360,mystery/thriller,a demo (drama/other),...,3,0,2,indoor,2,low,4,NaN,NaN,NaN
1,NaN,APPAL_7a,Appaloosa,7,00:01.70/41-00:31.73/761,30.03s,21x9,853x360,action/crime/drama (Western?),a demo (drama/other),...,4,3,0,outdoor,5,high,3,its hard to tell which characters are speaking...,NaN,NaN
2,NaN,STARD_2b,Stardust,2,04:05.52/5886-04:35.57/6607,30.04s,21x9,853x356,adventure/comedy/family (fantasy?),a demo (drama/other),...,3,3,2,outdoor,1,low,3,describe what the characters are doing,"candlelight picnic, candles on branches, champ...",NaN
3,1.0,Cloud_13a,Cloudy with a Chance of Meatballs,13,00:48.36/1160-01:18.38/1880,30.01s,16x9 (may be 1.77:1 to 1.85:1),853x480,animation/comedy/family,cartoon/animation,...,3,2,1,outdoor,4,high,3,describe the man's expressions and body langua...,NaN,NaN
4,2.0,Cloud_17a,Cloudy with a Chance of Meatballs,17,00:00.00/0-00:30.02/720,30.02s,16x9 (may be 1.77:1 to 1.85:1),853x480,animation/comedy/family,cartoon/animation,...,4,0,4,outdoor,4,high,3,describe the mayor,"extremely obese, white shirt, grey/blue jacket...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,202.0,BLUE_10a_POR,Blue Planet,10,00:00/00-00:30:00/922,30:00s,NaN,480x240,documentary,doc/nature,...,0,5,0,outdoor,4,high,4,NaN,NaN,NaN
205,203.0,NETW_7a_POR,Network,7,00:00/00-00:30:00/922,30:00s,NaN,480x240,drama/history,drama/other,...,5,0,5,indoor,3,medium,3,NaN,NaN,NaN
206,204.0,QUIZ_8a_POR,Quiz Show,8,00:00/00-00:29:00/893,29:00s,NaN,480x256,drama/history,drama/other,...,5,0,5,indoor,3,medium,3,NaN,NaN,NaN
207,205.0,SHAKE_11a_POR,Shakespeare in Love,11,00:00/00-00:30:00/910,30:00s,NaN,480x192,drama/romance,drama/other,...,4,3,1,outdoor,2,low,3,NaN,NaN,NaN


In [5]:
dist_nanmedian = subj2subj_dist["dist_nanmedian"]

In [6]:
genres = ['cartoon/animation', 'doc/nature', 'drama/other'] # data_info["genre category"].unique()
genres_label = ['animation', 'nature', 'drama']

In [7]:
video_names = subj2subj_dist["video_names"]

In [8]:
len(video_names)

200

In [9]:
subj2subj_dist_genres = {}
for gen in genres:
    subj2subj_dist_genres[gen] = []
    
for i, video_name in tqdm(enumerate(video_names)):
    dist_mat = subj2subj_dist[video_name]
    gen = data_info[data_info["file name"] == video_name]["genre category"].item()
    normalized_dist_mat = dist_mat / dist_nanmedian[i]
    subj2subj_dist_genres[gen].append(normalized_dist_mat)

for gen in genres:
    subj2subj_dist_genres[gen] = np.array(subj2subj_dist_genres[gen])

subj2subj_dist_mean_genres = {}
for gen in genres:
    subj2subj_dist_mean_genres[gen] = np.nanmean(subj2subj_dist_genres[gen], axis=0)

subj2subj_dist_mean_filled_genres = {}
for gen in genres:
    df = pd.DataFrame(data=subj2subj_dist_mean_genres[gen])
    df = df.fillna(df.median())
    subj2subj_fillnan = (df.values + df.values.T)/2
    subj2subj_dist_mean_filled_genres[gen] = subj2subj_fillnan

0it [00:00, ?it/s]

/tmp/ipykernel_26560/3161433254.py:16: RuntimeWarning: Mean of empty slice
  subj2subj_dist_mean_genres[gen] = np.nanmean(subj2subj_dist_genres[gen], axis=0)


In [10]:
gbvs_keys = ['All', 'Intensity', 'Orientation', 'Contrast', 'Flicker', 'Motion', 'DKL_Color']

In [11]:
subj2gbvs_dist_genres = {}
subj2gbvs_dist_mean_genres = {}
for gen in genres:
    subj2gbvs_dist_genres[gen] = {}
    subj2gbvs_dist_mean_genres[gen] = {}
    for gbvs_key in gbvs_keys:
        subj2gbvs_dist_genres[gen][gbvs_key] = []
    
for gbvs_key in gbvs_keys:
    for i, v in tqdm(enumerate(video_names)):
        dist_mat = subj2gbvs_dist[v].item()[gbvs_key]
        gen = data_info[data_info["file name"] == v]["genre category"].item()
        normalized_dist_mat = dist_mat / dist_nanmedian[i]
        subj2gbvs_dist_genres[gen][gbvs_key].append(normalized_dist_mat)

for gen in genres:
    for gbvs_key in gbvs_keys:
        dist_mat = np.array(subj2gbvs_dist_genres[gen][gbvs_key])
        subj2gbvs_dist_mean_genres[gen][gbvs_key] = np.nanmean(dist_mat, axis=0)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
training_methods = ["dino", "supervised"] 
depth_list = [4, 8, 12]
official_model_keys = ['dino_deit_small16', 'supervised_deit_small16']

In [13]:
subj2vit_dist_genres = {}
subj2vit_dist_mean_genres = {}
for gen in genres:
    subj2vit_dist_genres[gen] = {}
    subj2vit_dist_mean_genres[gen] = {}
    for tm in training_methods:
        subj2vit_dist_genres[gen][tm] = {}
        subj2vit_dist_mean_genres[gen][tm] = {}
        for depth in depth_list:
            subj2vit_dist_genres[gen][tm][str(depth)] = []
    for key in official_model_keys:
        subj2vit_dist_genres[gen][key] = []

In [14]:
for tm in training_methods:
    for depth in depth_list:
        for i, v in tqdm(enumerate(video_names)):
            dist_mat = subj2vit_dist[v].item()[tm][str(depth)]
            gen = data_info[data_info["file name"] == v]["genre category"].item()
            normalized_dist_mat = dist_mat / dist_nanmedian[i]
            subj2vit_dist_genres[gen][tm][str(depth)].append(normalized_dist_mat)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [15]:
for key in official_model_keys:
    for i, v in tqdm(enumerate(video_names)):
        dist_mat = subj2vit_official_dist[v].item()[key]
        gen = data_info[data_info["file name"] == v]["genre category"].item()
        normalized_dist_mat = dist_mat / dist_nanmedian[i]
        subj2vit_dist_genres[gen][key].append(normalized_dist_mat)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [16]:
subj2vit_dist_genres[gen].keys()

dict_keys(['dino', 'supervised', 'dino_deit_small16', 'supervised_deit_small16'])

In [17]:
for gen in genres:
    for tm in training_methods:
        for depth in depth_list:
            dist_mat = np.array(subj2vit_dist_genres[gen][tm][str(depth)])
            subj2vit_dist_mean_genres[gen][tm][str(depth)] = np.nanmean(dist_mat, axis=0)
    for key in official_model_keys:
        dist_mat = np.array(subj2vit_dist_genres[gen][key])
        subj2vit_dist_mean_genres[gen][key] = np.nanmean(dist_mat, axis=0)

official modelのNaNが取れない？

In [18]:
for g, gen in enumerate(genres):
    dist_mat_all_dict = {}
    # subj to subj
    dist_mat_all_dict["subj"] = subj2subj_dist_mean_filled_genres[gen]
    
    # subj to vit
    for tm in training_methods:
        for depth in depth_list:
            dist_mat_all_dict[f"{tm}_{str(depth)}"] = subj2vit_dist_mean_genres[gen][tm][str(depth)]
    for key in official_model_keys:
            dist_mat_all_dict[key] = subj2vit_dist_mean_genres[gen][key]
            
    # subj to gbvs
    subj2gbvs_mean_dist_concat = np.array([subj2gbvs_dist_mean_genres[gen][key] for key in gbvs_keys])
    dist_mat_all_dict["gbvs"] = subj2gbvs_mean_dist_concat.T
    print(dist_mat_all_dict.keys())
    savemat(f"../preprocessed_data/subj2subj_vit_gbvs_CW2019_{genres_label[g]}.mat", dist_mat_all_dict)

dict_keys(['subj', 'dino_4', 'dino_8', 'dino_12', 'supervised_4', 'supervised_8', 'supervised_12', 'dino_deit_small16', 'supervised_deit_small16', 'gbvs'])
dict_keys(['subj', 'dino_4', 'dino_8', 'dino_12', 'supervised_4', 'supervised_8', 'supervised_12', 'dino_deit_small16', 'supervised_deit_small16', 'gbvs'])
dict_keys(['subj', 'dino_4', 'dino_8', 'dino_12', 'supervised_4', 'supervised_8', 'supervised_12', 'dino_deit_small16', 'supervised_deit_small16', 'gbvs'])
